Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


In [2]:
insurance.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
insurance.isna().sum()
# Пропусков данных нет - супер :)

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [4]:
# Убираем строки-дубликаты
insurance.drop_duplicates(inplace=True)

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

#### Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Качество линейной регресси не должно измениться

**Обоснование:**  
Предположим что a = a', где a' - это предсказание, в которое мы передали матрицу признаков X, умноженную на матрицу
$$
a = Xw  
$$
$$
a' = X'w'
$$
$$
X' = XP
$$
Составим уравнение для формул предсказания для матрицы ХР   
$$
a' = (XP)((XP)^T(XP))^{-1} (XP)^Ty
$$
Раскрываем транспонирования по свойству: $(AB)^T = B^T A^T$
$$
a' = XP(P^TX^TXP)^{-1}P^TX^Ty
$$
раскроем скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$
$$
a' = XP(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty = X P P^{-1} (X^TX)^{-1} (P^T)^{-1} P^T X^T y
$$
В результате сокращения получим выражение
$$
a' = X E (X^T X) ^{-1} E X^T y = X(X^TX)^{-1}XTy
$$
Т.к. $a = X(X^T X)^{-1} X^T y$ можно утверждать что $a' = a$ а $w' = w$

## 3. Алгоритм преобразования

**Алгоритм**

Умножаем матрицу признаков на некоторую обратимую матрицу, что не приведет к изменению качества линейной регрессии

**Обоснование**

Расчеты выполненные в шаге 2.

## 4. Проверка алгоритма

In [5]:
# Выделяем нецелевые и целевые признаки из исходного датасета
X_source = insurance.drop(['Страховые выплаты'], axis=1)
y_source = insurance['Страховые выплаты']

In [6]:
# Создаем случайную обратимую матрицу 
# Используем np.linalg.inv чтобы убедиться что матрица обратима
P = np.linalg.inv(np.random.normal(loc =0, scale =1, size=(X_source.shape[1], X_source.shape[1])))

In [7]:
# Умножаем матрицу признаков на новую обратимую матрицу
XP = X_source @ P

In [8]:
# Проверяем что размеры матриц X и XP идентичны
X_source.shape, XP.shape

((4847, 4), (4847, 4))

In [9]:
# создаем модель линейной регрессии
model = LinearRegression()

In [10]:
# Делим исходные признаки на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_source, y_source, test_size=0.2, random_state=12345)

In [11]:
# Делим признаки после умножения на обратимую матрицу на обучающую и тестовую выборки
XP_train, XP_test, y_train, y_test = train_test_split(XP, y_source, test_size=0.2, random_state=12345)

In [12]:
model.fit(X_train, y_train)
sourced_predict = model.predict(X_test)
print(f'R2_score предсказания после обучения на исходных признаках: {r2_score(y_test, sourced_predict)}')

R2_score предсказания после обучения на исходных признаках: 0.41605492161510915


In [13]:
model.fit(XP_train, y_train)
inv_predict = model.predict(XP_test)
print(f'R2_score предсказания после обучения на признаках, полученных после умножения на обратимую матрицу: {r2_score(y_test, inv_predict)}')

R2_score предсказания после обучения на признаках, полученных после умножения на обратимую матрицу: 0.4160549216142152


### Вывод:  
Показатели R2 у предсказаний модели, обученной на исходных признаках и у модели, обученной на признаках, полученных после умножения на обратимую матрицу практически идентичны (Разница появляется лишь после 11го знака после запятой). Соответственно алгоритм защиты персональных данных путём умножения признаков на обратимую матрицу можно использовать в работе.